# Actividad - Proyecto práctico


> La actividad se desarrollará en grupos pre-definidos de 2-3 alumnos. Se debe indicar los nombres en orden alfabético (de apellidos). Recordad que esta actividad se corresponde con un 30% de la nota final de la asignatura. Se debe entregar entregar el trabajo en la presente notebook.
*   Alumno 1:
*   Alumno 2:
*   Alumno 3:






---
## **PARTE 1** - Instalación y requisitos previos

> Las prácticas han sido preparadas para poder realizarse en el entorno de trabajo de Google Colab. Sin embargo, esta plataforma presenta ciertas incompatibilidades a la hora de visualizar la renderización en gym. Por ello, para obtener estas visualizaciones, se deberá trasladar el entorno de trabajo a local. Por ello, el presente dosier presenta instrucciones para poder trabajar en ambos entornos. Siga los siguientes pasos para un correcto funcionamiento:
1.   **LOCAL:** Preparar el enviroment, siguiendo las intrucciones detalladas en la sección *1.1.Preparar enviroment*.
2.  **AMBOS:** Modificar las variables "mount" y "drive_mount" a la carpeta de trabajo en drive en el caso de estar en Colab, y ejecturar la celda *1.2.Localizar entorno de trabajo*.
3. **COLAB:** se deberá ejecutar las celdas correspondientes al montaje de la carpeta de trabajo en Drive. Esta corresponde a la sección *1.3.Montar carpeta de datos local*.
4.  **AMBOS:** Instalar las librerías necesarias, siguiendo la sección *1.4.Instalar librerías necesarias*.


---
### 1.1. Preparar enviroment (solo local)



> Para preparar el entorno de trabajo en local, se han seguido los siguientes pasos:
1. En Windows, puede ser necesario instalar las C++ Build Tools. Para ello, siga los siguientes pasos: https://towardsdatascience.com/how-to-install-openai-gym-in-a-windows-environment-338969e24d30.
2. Instalar Anaconda
3. Siguiendo el código que se presenta comentado en la próxima celda: Crear un enviroment, cambiar la ruta de trabajo, e instalar librerías básicas.


```
conda create --name miar_rl python=3.8
conda activate miar_rl
cd "PATH_TO_FOLDER"
conda install git
pip install jupyter
```


4. Abrir la notebook con *jupyter-notebook*.



```
jupyter-notebook
```


---
### 1.4. Instalar librerías necesarias
(descomenta si es la primera vez que corres el codigo)

In [5]:
# %pip install gym==0.17.3
# %pip install git+https://github.com/Kojoley/atari-py.git
# %pip install pyglet==1.5.0
# %pip install h5py==3.1.0
# %pip install Pillow==9.5.0
# %pip install keras-rl2==1.0.5
# %pip install Keras==2.2.4
# %pip install tensorflow==2.5.3
# %pip install torch==2.0.1
# %pip install agents==1.4.0

---
## **PARTE 2**. Enunciado

Consideraciones a tener en cuenta:

- El entorno sobre el que trabajaremos será _SpaceInvaders-v0_ y el algoritmo que usaremos será _DQN_.

- Para nuestro ejercicio, el requisito mínimo será alcanzado cuando el agente consiga una **media de recompensa por encima de 20 puntos en modo test**. Por ello, esta media de la recompensa se calculará a partir del código de test en la última celda del notebook.

Este proyecto práctico consta de tres partes:

1.   Implementar la red neuronal que se usará en la solución
2.   Implementar las distintas piezas de la solución DQN
3.   Justificar la respuesta en relación a los resultados obtenidos

**Rúbrica**: Se valorará la originalidad en la solución aportada, así como la capacidad de discutir los resultados de forma detallada. El requisito mínimo servirá para aprobar la actividad, bajo premisa de que la discusión del resultado sera apropiada.

IMPORTANTE:

* Si no se consigue una puntuación óptima, responder sobre la mejor puntuación obtenida.
* Para entrenamientos largos, recordad que podéis usar checkpoints de vuestros modelos para retomar los entrenamientos. En este caso, recordad cambiar los parámetros adecuadamente (sobre todo los relacionados con el proceso de exploración).
* Se deberá entregar unicamente el notebook y los pesos del mejor modelo en un fichero .zip, de forma organizada.
* Cada alumno deberá de subir la solución de forma individual.

---
## **PARTE 3**. Desarrollo y preguntas

#### Importar librerías

In [1]:
from __future__ import division

from PIL import Image
import numpy as np
import gym

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Activation, Flatten, Convolution2D, Permute, Input, concatenate, Lambda
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

c:\Users\javie\anaconda3\envs\gym\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\javie\anaconda3\envs\gym\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
c:\Users\javie\anaconda3\envs\gym\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
class StickyActionEnv(gym.Wrapper):
    def __init__(self, env, sticky_prob=0.25):
        super(StickyActionEnv, self).__init__(env)
        self.sticky_prob = sticky_prob
        self.last_action = None

    def step(self, action):
        if self.last_action is not None and np.random.rand() < self.sticky_prob:
            action = self.last_action
        self.last_action = action
        obs, reward, done, info = self.env.step(action)
        return obs, reward, done, info

    def reset(self, **kwargs):
        self.last_action = None
        return self.env.reset(**kwargs)

#### Configuración base

In [3]:
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4
IN_COLAB=False

env_name = 'SpaceInvaders-v0'
env = gym.make(env_name)
sticky_env = StickyActionEnv(env, sticky_prob=0.25)
np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

In [4]:
print("Numero de acciones disponibles:" + str(nb_actions))
print("Formato de las observaciones:")
sticky_env.observation_space

Numero de acciones disponibles:6
Formato de las observaciones:


Box(0, 255, (210, 160, 3), uint8)

Un espacio de soluciones de dimensiones (210,160,3) tiene un tamaño inmanejable. Se puede reducir ese espacio a un tamaño de 84x84 con una window length 4.

In [5]:
class AtariProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')

    def process_state_batch(self, batch):
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)

## 1. Implementación de la red neuronal

Se puede hacer:


1.   DQN
2.   Doble DQN
3.   Duel DQN

Y luego también variar el discount factor, cambios en epsilon, tamaño de memoria, uso de epsilon greedy vs. boltzman y escribir como varía el resultado. Cambiar el número de episodios es un poco gilipollez, dara mejores resultados por más train. Usamos un número pequeño de episodios. el mínimo para que llegue a 20 puntos y a partir de ahí vamos probando. También podríamos cambiar el input de la red neuronal.

## VER EN CONJUNTO





In [6]:
# arquitectura de DeepMind paper
def construct_double_q_network(input_shape, num_actions, loss='mse', optimizer=Adam(lr=0.0001)):

        input_layer = Input(shape=input_shape)
        permuted_layer = Permute((3, 2, 1))(input_layer)   #IMPORTANTE 
        conv1 = Convolution2D(32, (8, 8), strides=(4, 4), activation='relu')(permuted_layer)
        conv2 = Convolution2D(64, (4, 4), strides=(2, 2), activation='relu')(conv1)
        conv3 = Convolution2D(64, (3, 3), activation='relu')(conv2)
        flatten = Flatten()(conv3)

        fc1 = Dense(512)(flatten)
        advantage = Dense(num_actions)(fc1)

        fc2 = Dense(512)(flatten)
        value = Dense(1)(fc2)

        policy = concatenate([advantage, value], axis=-1)
        #policy = Lambda(lambda x: x[:, :-1] - K.mean(x[:, :-1], axis=1, keepdims=True) + x[:, -1])(policy)
        def custom_lambda(x):
            adv = x[:, :-1]
            val = x[:, -1:]
            print(f"adv shape: {adv.shape}, val shape: {val.shape}")
            return adv - K.mean(adv, axis=1, keepdims=True) + val

        policy = Lambda(custom_lambda)(policy)


        model = Model(inputs=input_layer, outputs=policy)
        model.compile(loss=loss, optimizer=optimizer)


        print("Successfully constructed networks.")

        return model

c:\Users\javie\anaconda3\envs\gym\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Ver aquí parámetros del https://www.tensorflow.org/agents/api_docs/python/tf_agents/agents/DqnAgent

In [10]:
memory = SequentialMemory(limit=50000, window_length=WINDOW_LENGTH)
processor = AtariProcessor()
# Definir la política base
base_policy = EpsGreedyQPolicy()

# Configurar LinearAnnealedPolicy con los parámetros necesarios
policy = LinearAnnealedPolicy(base_policy,
                              attr='eps',  # El atributo que se debe anillar
                              value_max=1.0,  # Valor inicial de epsilon
                              value_min=0.1,  # Valor final de epsilon
                              value_test=0.05,  # Valor de epsilon durante la prueba
                              nb_steps=10000)  # Número de pasos para el anillado

In [12]:
model = construct_double_q_network(input_shape=(4, 84,84), num_actions=6, loss='mse', optimizer=Adam(lr=0.0001))
model.summary()


adv shape: (None, 6), val shape: (None, 1)
Successfully constructed networks.
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 4, 84, 84)]  0                                            
__________________________________________________________________________________________________
permute_2 (Permute)             (None, 84, 84, 4)    0           input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 20, 20, 32)   8224        permute_2[0][0]                  
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 9, 9, 64)     32832       conv2d_6[0][0]                   
______________

In [9]:
# dqn = DQNAgent(model=model, 
#                 nb_actions=nb_actions, 
#                 policy=policy,
#                memory=memory, 
#                processor=processor,
#                nb_steps_warmup=50000, 
#                gamma=.99,
#                target_model_update=10000,
#                train_interval=4)




dqn = DQNAgent(model=model, 
                nb_actions=nb_actions,
                processor=processor,
                memory=memory,
                nb_steps_warmup=50000,
                target_model_update=1e-2, 
                policy=policy, 
                enable_double_dqn=True) #target model te congela durante 100 steps lo que se usa para el "ground truth"

dqn.compile(Adam(learning_rate=.00025), metrics=['mae'])

adv shape: (None, 6), val shape: (None, 1)


In [11]:
# Training part
weights_filename = 'dqn3_{}_weights.h5f'.format(env_name)
checkpoint_weights_filename = 'dqn3_' + env_name + '_weights_{step}.h5f'
log_filename = 'dqn3_{}_log.json'.format(env_name)
callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=250000)]
callbacks += [FileLogger(log_filename, interval=100)]

dqn.fit(sticky_env, callbacks=callbacks, nb_steps=1000000, log_interval=10000, visualize=False)

dqn.save_weights(weights_filename, overwrite=True)

Training for 1000000 steps ...
Interval 1 (0 steps performed)


c:\Users\javie\anaconda3\envs\gym\lib\site-packages\tensorflow\python\keras\engine\training.py:2424: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


 2129/10000 [=====>........................] - ETA: 1:03 - reward: 0.016done, took 17.490 seconds


In [16]:
# Testing part to calculate the mean reward
weights_filename = 'dqn2_{}_weights_250000.h5f'.format(env_name)
dqn.load_weights(weights_filename)
dqn.test(env, nb_episodes=10, visualize=False)

Testing for 10 episodes ...
Episode 1: reward: 16.000, steps: 731
Episode 2: reward: 6.000, steps: 706
Episode 3: reward: 17.000, steps: 1109
Episode 4: reward: 12.000, steps: 729
Episode 5: reward: 14.000, steps: 703
Episode 6: reward: 6.000, steps: 432
Episode 7: reward: 12.000, steps: 697
Episode 8: reward: 10.000, steps: 666
Episode 9: reward: 18.000, steps: 1103
Episode 10: reward: 20.000, steps: 1065


In [14]:
checkpoint_filename = 'dqn2_{}_weights_250000.h5f'.format(env_name)
dqn.load_weights(checkpoint_filename)

weights_filename = 'dqn2_{}_weights.h5f'.format(env_name)
checkpoint_weights_filename = 'dqn2_' + env_name + '_weights_{step}.h5f'
log_filename = 'dqn2_{}_log.json'.format(env_name)
callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=250000)]
callbacks += [FileLogger(log_filename, interval=100)]

In [18]:
dqn.fit(env, callbacks=callbacks, nb_steps=750000, log_interval=10000, visualize=False)

dqn.save_weights(weights_filename, overwrite=True)

Training for 750000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 49s - reward: 0.0000e+00

10000/10000 [==============================] - 58s 6ms/step - reward: 0.0157
13 episodes - episode_reward: 11.538 [4.000, 24.000] - ale.lives: 1.990

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 58s 6ms/step - reward: 0.0169
13 episodes - episode_reward: 13.308 [4.000, 21.000] - ale.lives: 1.996

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 58s 6ms/step - reward: 0.0179
15 episodes - episode_reward: 11.800 [6.000, 23.000] - ale.lives: 2.115

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 57s 6ms/step - reward: 0.0169
13 episodes - episode_reward: 13.077 [4.000, 23.000] - ale.lives: 2.059

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 57s 6ms/step - reward: 0.0157
13 episodes - episode_reward: 12.154 [5.000, 23.000] - ale.lives: 2.215

Interval 6 (50000 steps performed)
10000/10000 [==============================] - 1448s 145ms/step - reward: 0

In [18]:
dqn.load_weights(weights_filename)
dqn.test(env, nb_episodes=10, visualize=True)

Testing for 10 episodes ...
Episode 1: reward: 6.000, steps: 568
Episode 2: reward: 21.000, steps: 846
Episode 3: reward: 14.000, steps: 904
Episode 4: reward: 17.000, steps: 1220
Episode 5: reward: 13.000, steps: 705
Episode 6: reward: 11.000, steps: 680
Episode 7: reward: 14.000, steps: 935
Episode 8: reward: 23.000, steps: 1079
Episode 9: reward: 16.000, steps: 940
Episode 10: reward: 15.000, steps: 899


: 

3. Justificación de los parámetros seleccionados y de los resultados obtenidos

---